# One File With All the Dissertation Data Preprocessing

In [3]:
%matplotlib inline
from scipy.io import loadmat
import numpy as np
import pandas as pd
from pathlib import Path
import re
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from scipy.signal import medfilt, butter, filtfilt
from sklearn import preprocessing
import math

In [12]:
# import the respective datasets
# USC-HAD
def import_USC_HAD_data(activity, subject, trial):
    path = f'Data/USC-HAD/Subject{subject}/a{activity}t{trial}.mat'
    if Path(path).is_file():
        data = loadmat(path)
        return data
    else:
        return None
# UTD-MHAD
def import_UTD_MHAD_data(activity, subject, trial, data_type):
    path = f'Data/UTD MHAD/Inertial/a{activity}_s{subject}_t{trial}_{data_type}.mat'
    if Path(path).is_file():
        data = loadmat(path)
        return data
    else:
        return None
# UCI-HAR (HAPT)
def import_UCI_HAR_data(subject, trial, data_type):
    path = f'Data/UCI-HAR/RawData/{data_type}_exp{trial}_user{subject}.txt'
    if Path(path).is_file():
        data = np.loadtxt(path)
        data = np.array(data)
        return data
    else:
        return None

In [13]:
# functions to transform the wearable sensor data of the respective datasets
# USC-HAD
def transform_inertial_USC_HAD_data(activity, subject, trial):
    data = import_USC_HAD_data(activity, subject, trial)
    if data is None:return None
    inertial_data = data['sensor_readings']
    result = np.insert(inertial_data, 0, [[activity], [subject], [trial]], axis=1)
    result = np.array(result)
    return result
#UTD-MHAD
def transform_inertial_UTD_MHAD_data(activity, subject, trial):
    data = import_UTD_MHAD_data(activity, subject, trial, data_type='inertial')
    if data is None:return None
    inertial_data = data['d_iner']
    result = np.insert(inertial_data, 0, [[activity], [subject], [trial]], axis=1)
    result = np.array(result)
    return result

# functions to convert the tranformed wearable sensor data into a DataFrame
# USC-HAD
def inertial_USC_HAD_data_df(activity, subject, trial):
    data = transform_inertial_USC_HAD_data(activity, subject, trial)
    if data is None:return None
    df = pd.DataFrame(data)
    df.columns = ['activity', 'subject', 'trial', 'x-acc', 'y-acc', 'z-acc', 'x-gyro', 'y-gyro', 'z-gyro']
    return df
# UTD-HAD
def inertial_UTD_MHAD_data_df(activity, subject, trial):
    data = transform_inertial_UTD_MHAD_data(activity, subject, trial)
    if data is None:return None
    df = pd.DataFrame(data)
    df.columns = ['activity', 'subject', 'trial', 'x-acc', 'y-acc', 'z-acc', 'x-gyro', 'y-gyro', 'z-gyro']
    return df
# HAPT (UCI-HAR)
def inertial_UCI_HAR_data_df(subject, trial):
    data_acc = import_UCI_HAR_data(subject, trial, data_type='acc')
    data_gyro = import_UCI_HAR_data(subject, trial, data_type='gyro')
    if data_acc is None:return None
    data_acc_st = np.insert(data_acc, 0, [[subject], [trial]], axis=1)
    df1 = pd.DataFrame(data_acc_st)
    df2 = pd.DataFrame(data_gyro)
    df = pd.concat([df1, df2], axis=1)
    df.columns = ['subject', 'trial', 'x-acc', 'y-acc', 'z-acc', 'x-gyro', 'y-gyro', 'z-gyro']
    return df

In [14]:
# noise reduction

# the units for the accelerometer readings are m/s-2
# the units for the gyroscope readings are s-1
def sensor_image(raw_data, frequency):
    # calulating the time length of the plot
    time = [1/(frequency) * t for t in range(len(raw_data))]
    # creating the plot
    fig = plt.figure(figsize=(12,4))
    plt.plot(time,raw_data)
    plt.xlabel('Time (seconds)', fontsize=18)
    plt.ylabel('Sensor Reading', fontsize=18)
    plt.ylim([0,6])
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.grid(color='grey', linestyle='--', linewidth=0.2)
    plt.tight_layout()
    plt.savefig('sensor_image.png')

In [15]:
cutoff= 20  # the cutoff frequency of the filter
def butterworth_filter(raw_data,frequency):
    W = cutoff/(frequency/2)
    # Preparing the low-pass filter
    # third order filter
    b, a = butter(3, W, 'low')
    # Using a filter on the raw data
    output = filtfilt(b, a, raw_data)
    # Reutrning filtered raw data
    return output

In [16]:
# build big dataframes representing the datasets
# whilst applying the noise filtering

USC_HAD_df = None
for activity in range(1,13):
    for subject in range(1,15):
        for trial in range(1,6):
            data = inertial_USC_HAD_data_df(activity, subject, trial)
            data_filt = pd.DataFrame()
            for label in data.columns[3:9]:
                data_filt[label] = butterworth_filter(data[label],100)
            data_filt = pd.concat([data['activity'], data['subject'], data['trial'], data_filt], axis=1)
            USC_HAD_df = pd.concat([USC_HAD_df, data_filt])

UTD_MHAD_df = None
for activity in range(1,22):
    for subject in range(1,9):
        for trial in range(1,5):
            data = inertial_UTD_MHAD_data_df(activity, subject, trial)
            if data is None: continue # has three missing observations
            data_filt = pd.DataFrame()
            for label in data.columns[3:9]:
                data_filt[label] = butterworth_filter(data[label],50)
            data_filt = pd.concat([data['activity'], data['subject'], data['trial'], data_filt], axis=1)
            UTD_MHAD_df = pd.concat([UTD_MHAD_df, data_filt])
            
UCI_HAR_df = None
for subject in range(1,31):
    for trial in range(1,62):
        data = inertial_UCI_HAR_data_df(subject, trial)
        if data is None: continue # files are named such that this is required
        data_filt = pd.DataFrame()
        for label in data.columns[2:8]:
            data_filt[label] = butterworth_filter(data[label],50)
        data_filt = pd.concat([data['subject'], data['trial'], data_filt], axis=1)
        UCI_HAR_df = pd.concat([UCI_HAR_df, data])

In [17]:
# normalisation
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))

# USC-HAD
USC_act = USC_HAD_df['activity']
USC_sub = USC_HAD_df['subject']
USC_tr = USC_HAD_df['trial']
USC_x_a = USC_HAD_df['x-acc']
USC_y_a = USC_HAD_df['y-acc']
USC_z_a = USC_HAD_df['z-acc']
USC_x_g = USC_HAD_df['x-gyro']
USC_y_g = USC_HAD_df['y-gyro']
USC_z_g = USC_HAD_df['z-gyro']

USC_act = USC_HAD_df['activity'].values.reshape((len(USC_act),1))
USC_sub = USC_HAD_df['subject'].values.reshape((len(USC_sub),1))
USC_tr = USC_HAD_df['trial'].values.reshape((len(USC_tr),1))
USC_x_a = USC_HAD_df['x-acc'].values.reshape((len(USC_x_a),1))
USC_y_a = USC_HAD_df['y-acc'].values.reshape((len(USC_y_a),1))
USC_z_a = USC_HAD_df['z-acc'].values.reshape((len(USC_z_a),1))
USC_x_g = USC_HAD_df['x-gyro'].values.reshape((len(USC_x_g),1))
USC_y_g = USC_HAD_df['y-gyro'].values.reshape((len(USC_y_g),1))
USC_z_g = USC_HAD_df['z-gyro'].values.reshape((len(USC_z_g),1))

USC_x_a_scaled = min_max_scaler.fit_transform(USC_x_a)
USC_y_a_scaled = min_max_scaler.fit_transform(USC_y_a)
USC_z_a_scaled = min_max_scaler.fit_transform(USC_z_a)
USC_x_g_scaled = min_max_scaler.fit_transform(USC_x_g)
USC_y_g_scaled = min_max_scaler.fit_transform(USC_y_g)
USC_z_g_scaled = min_max_scaler.fit_transform(USC_z_g)

USC_scaled_data = np.concatenate((USC_act,USC_sub,USC_tr,USC_x_a_scaled,USC_y_a_scaled,USC_z_a_scaled,USC_x_g_scaled,USC_y_g_scaled,USC_z_g_scaled), axis=1)
USC_HAD_df = pd.DataFrame(USC_scaled_data)
USC_HAD_df.columns = ['activity', 'subject', 'trial', 'x-acc', 'y-acc', 'z-acc', 'x-gyro', 'y-gyro', 'z-gyro']

# UTD-MHAD
UTD_act = UTD_MHAD_df['activity']
UTD_sub = UTD_MHAD_df['subject']
UTD_tr = UTD_MHAD_df['trial']
UTD_x_a = UTD_MHAD_df['x-acc']
UTD_y_a = UTD_MHAD_df['y-acc']
UTD_z_a = UTD_MHAD_df['z-acc']
UTD_x_g = UTD_MHAD_df['x-gyro']
UTD_y_g = UTD_MHAD_df['y-gyro']
UTD_z_g = UTD_MHAD_df['z-gyro']

UTD_act = UTD_MHAD_df['activity'].values.reshape((len(UTD_act),1))
UTD_sub = UTD_MHAD_df['subject'].values.reshape((len(UTD_sub),1))
UTD_tr = UTD_MHAD_df['trial'].values.reshape((len(UTD_tr),1))
UTD_x_a = UTD_MHAD_df['x-acc'].values.reshape((len(UTD_x_a),1))
UTD_y_a = UTD_MHAD_df['y-acc'].values.reshape((len(UTD_y_a),1))
UTD_z_a = UTD_MHAD_df['z-acc'].values.reshape((len(UTD_z_a),1))
UTD_x_g = UTD_MHAD_df['x-gyro'].values.reshape((len(UTD_x_g),1))
UTD_y_g = UTD_MHAD_df['y-gyro'].values.reshape((len(UTD_y_g),1))
UTD_z_g = UTD_MHAD_df['z-gyro'].values.reshape((len(UTD_z_g),1))

UTD_x_a_scaled = min_max_scaler.fit_transform(UTD_x_a)
UTD_y_a_scaled = min_max_scaler.fit_transform(UTD_y_a)
UTD_z_a_scaled = min_max_scaler.fit_transform(UTD_z_a)
UTD_x_g_scaled = min_max_scaler.fit_transform(UTD_x_g)
UTD_y_g_scaled = min_max_scaler.fit_transform(UTD_y_g)
UTD_z_g_scaled = min_max_scaler.fit_transform(UTD_z_g)

UTD_scaled_data = np.concatenate((UTD_act,UTD_sub,UTD_tr,UTD_x_a_scaled,UTD_y_a_scaled,UTD_z_a_scaled,UTD_x_g_scaled,UTD_y_g_scaled,UTD_z_g_scaled), axis=1)
UTD_MHAD_df = pd.DataFrame(UTD_scaled_data)
UTD_MHAD_df.columns = ['activity', 'subject', 'trial', 'x-acc', 'y-acc', 'z-acc', 'x-gyro', 'y-gyro', 'z-gyro']

# UCI-HAR
UCI_sub = UCI_HAR_df['subject']
UCI_tr = UCI_HAR_df['trial']
UCI_x_a = UCI_HAR_df['x-acc']
UCI_y_a = UCI_HAR_df['y-acc']
UCI_z_a = UCI_HAR_df['z-acc']
UCI_x_g = UCI_HAR_df['x-gyro']
UCI_y_g = UCI_HAR_df['y-gyro']
UCI_z_g = UCI_HAR_df['z-gyro']

UCI_sub = UCI_HAR_df['subject'].values.reshape((len(UCI_sub),1))
UCI_tr = UCI_HAR_df['trial'].values.reshape((len(UCI_tr),1))
UCI_x_a = UCI_HAR_df['x-acc'].values.reshape((len(UCI_x_a),1))
UCI_y_a = UCI_HAR_df['y-acc'].values.reshape((len(UCI_y_a),1))
UCI_z_a = UCI_HAR_df['z-acc'].values.reshape((len(UCI_z_a),1))
UCI_x_g = UCI_HAR_df['x-gyro'].values.reshape((len(UCI_x_g),1))
UCI_y_g = UCI_HAR_df['y-gyro'].values.reshape((len(UCI_y_g),1))
UCI_z_g = UCI_HAR_df['z-gyro'].values.reshape((len(UCI_z_g),1))

UCI_x_a_scaled = min_max_scaler.fit_transform(UCI_x_a)
UCI_y_a_scaled = min_max_scaler.fit_transform(UCI_y_a)
UCI_z_a_scaled = min_max_scaler.fit_transform(UCI_z_a)
UCI_x_g_scaled = min_max_scaler.fit_transform(UCI_x_g)
UCI_y_g_scaled = min_max_scaler.fit_transform(UCI_y_g)
UCI_z_g_scaled = min_max_scaler.fit_transform(UCI_z_g)

UCI_scaled_data = np.concatenate((UCI_sub,UCI_tr,UCI_x_a_scaled,UCI_y_a_scaled,UCI_z_a_scaled,UCI_x_g_scaled,UCI_y_g_scaled,UCI_z_g_scaled), axis=1)
UCI_HAR_df = pd.DataFrame(UCI_scaled_data)
UCI_HAR_df.columns = ['subject', 'trial', 'x-acc', 'y-acc', 'z-acc', 'x-gyro', 'y-gyro', 'z-gyro']

In [8]:
# the UCI_HAR dataset required further preprocessing to correctly label the sensor signals and remove any unlabelled data

# import the UCI activity labels
path = f'Data/UCI-HAR/RawData/labels.txt'
labels = np.loadtxt(path)
labels_df = pd.DataFrame(labels)

# calculates the during of each activity
act_range = []
for subject in range(1,31):
    for trial in range(1,62):
        sub = UCI_HAR_df[UCI_HAR_df['subject'] == subject]
        sub_tr = sub[sub['trial'] == trial]
        if sub_tr.empty is not True:
            act_range.append(len(sub_tr))
            
# the following code removes the unlabelled data and appropriately labels the remaining data
UCI_HAR_df_act = None
act_labels= {'activity': []}
act_labels_df = pd.DataFrame(data=act_labels)
act_start_point = 0
trial_count = 0
for activity in range(0,len(labels_df)-1):
    act = pd.DataFrame()
    act_start = np.int(labels_df.iloc[activity,3])
    act_end = np.int(labels_df.iloc[activity,4])
    act_diff = act_end-act_start+1
    UCI_HAR_df_act = pd.concat([UCI_HAR_df_act, UCI_HAR_df.iloc[act_start+act_start_point:act_end+act_start_point+1,:]])
    act['activity'] = [labels_df.iloc[activity,2]] * act_diff
    act_labels_df = pd.concat([act_labels_df, act], axis=0, ignore_index=False)
    if labels_df.iloc[activity+1,3] < labels_df.iloc[activity,3]:
        act_start_point = act_start_point + act_range[trial_count]
        trial_count = trial_count +1
# add the final activity
UCI_HAR_df_act = pd.concat([UCI_HAR_df_act, UCI_HAR_df.iloc[np.int(labels_df.iloc[len(labels_df)-1,3])+act_start_point:np.int(labels_df.iloc[len(labels_df)-1,4])+act_start_point+1,:]])
act = pd.DataFrame()
act['activity'] = [labels_df.iloc[len(labels_df)-1,2]] * (((np.int(labels_df.iloc[len(labels_df)-1,4]))-(np.int(labels_df.iloc[len(labels_df)-1,3])))+1)
act_labels_df = pd.concat([act_labels_df, act], axis=0, ignore_index=True)
# UCI_HAR_df_proc = pd.concat([act_labels_df,UCI_HAR_df_act], axis=1, ignore_index=True)

<ipython-input-8-55a49376c19d>:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  act_start = np.int(labels_df.iloc[activity,3])
<ipython-input-8-55a49376c19d>:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy

In [11]:
#UCI_HAR_df_act.to_csv('UCI_HAR_df_act.csv')
#act_labels_df.to_csv('act_labels_df.csv')

KeyboardInterrupt: 

In [18]:
# since merging the dataframe above took long to run, we save it and recall it
UCI_HAR_df_proc = pd.read_csv(r'Processed_UCI_HAR.csv')
# remove the transitional activities
UCI_HAR_df_proc = UCI_HAR_df_proc.loc[UCI_HAR_df_proc['activity'].isin([1,2,3,4,5,6])]

In [19]:
# a function to split wearable sensor data into time windows, given the dataset and its frequency
# the time window is in seconds and the overlap is a percentage
def sliding_windows(window_length, overlap, data, frequency):
    window = []
    window_data = []
    labels = []
    sensor_readings = int(window_length * frequency)
    diff = int(sensor_readings * (100 - overlap) / 100)
    for i in range(0, len(data), diff):
        x_acc = data['x-acc'].values[i:i+sensor_readings]
        y_acc = data['y-acc'].values[i:i+sensor_readings]
        z_acc = data['z-acc'].values[i:i+sensor_readings]
        x_gyro = data['x-gyro'].values[i:i+sensor_readings]
        y_gyro = data['y-gyro'].values[i:i+sensor_readings]
        z_gyro = data['z-gyro'].values[i:i+sensor_readings]
        window.append([x_acc,y_acc,z_acc,x_gyro,y_gyro,z_gyro])
        window_arr = np.asarray(window)
        # only include windows that have the desired number of sensor readings
        if window_arr.shape[2] == sensor_readings:
            label = data['activity'].values[i]
            labels.append(label)
            window_arr.reshape(sensor_readings, 6)
            window_data.append(window_arr)
            window = []
        else:
            window = []
        
    window_data = np.asarray(window_data)
    labels = np.asarray(labels)
    # reshape into a square image
    window_data = window_data.reshape(window_data.shape[0],int(math.sqrt(sensor_readings)),int(math.sqrt(sensor_readings)),6)
    
    return window_data, labels

In [20]:
# a function to sort the data into time windows and split these into test and training sets of the features and labels
def train_test(window_length, overlap, data, frequency):
    features, labels = sliding_windows(window_length, overlap, data, frequency)
    train_features, test_features, train_labels, test_labels  = train_test_split(features, labels, test_size=0.2)
    return train_features, test_features, train_labels, test_labels

In [21]:
# training and test sets for the three datasets and for different sample generation parameters
# overlap of 25%, 50% and 75%
# window length of 1, 1.5, 2 and 2.5 seconds
# 12 different sample parameter configurations
# 12x3=36 datasets in total

# USC-HAD (100Hz)
# 1 second window size, 25% overlap
USC_features_train_1_sec_25ol, USC_features_test_1_sec_25ol, USC_labels_train_1_sec_25ol, USC_labels_test_1_sec_25ol=\
train_test(1, 25, USC_HAD_df, 100)
# 1 second window size, 50% overlap
USC_features_train_1_sec_50ol, USC_features_test_1_sec_50ol, USC_labels_train_1_sec_50ol, USC_labels_test_1_sec_50ol =\
train_test(1, 50, USC_HAD_df, 100)
# 1 second window size, 75% overlap
USC_features_train_1_sec_75ol, USC_features_test_1_sec_75ol, USC_labels_train_1_sec_75ol, USC_labels_test_1_sec_75ol =\
train_test(1, 75, USC_HAD_df, 100)
# 1.5 second window size, 25% overlap
USC_features_train_1_5_sec_25ol, USC_features_test_1_5_sec_25ol, USC_labels_train_1_5_sec_25ol, USC_labels_test_1_5_sec_25ol\
= train_test(1.44, 25, USC_HAD_df, 100)
# 1.5 second window size, 50% overlap
USC_features_train_1_5_sec_50ol, USC_features_test_1_5_sec_50ol, USC_labels_train_1_5_sec_50ol, USC_labels_test_1_5_sec_50ol\
= train_test(1.44, 50, USC_HAD_df, 100)
# 1.5 second window size, 75% overlap
USC_features_train_1_5_sec_75ol, USC_features_test_1_5_sec_75ol, USC_labels_train_1_5_sec_75ol, USC_labels_test_1_5_sec_75ol\
= train_test(1.44, 75, USC_HAD_df, 100)
# 2 second window size, 25% overlap
USC_features_train_2_sec_25ol, USC_features_test_2_sec_25ol, USC_labels_train_2_sec_25ol, USC_labels_test_2_sec_25ol =\
train_test(1.96, 25, USC_HAD_df, 100)
# 2 second window size, 50% overlap
USC_features_train_2_sec_50ol, USC_features_test_2_sec_50ol, USC_labels_train_2_sec_50ol, USC_labels_test_2_sec_50ol =\
train_test(1.96, 50, USC_HAD_df, 100)
# 2 second window size, 75% overlap
USC_features_train_2_sec_75ol, USC_features_test_2_sec_75ol, USC_labels_train_2_sec_75ol, USC_labels_test_2_sec_75ol =\
train_test(1.96, 75, USC_HAD_df, 100)
# 2.5 second window size, 25% overlap
USC_features_train_2_5_sec_25ol, USC_features_test_2_5_sec_25ol, USC_labels_train_2_5_sec_25ol, USC_labels_test_2_5_sec_25ol\
= train_test(2.56, 25, USC_HAD_df, 100)
# 2.5 second window size, 50% overlap
USC_features_train_2_5_sec_50ol, USC_features_test_2_5_sec_50ol, USC_labels_train_2_5_sec_50ol, USC_labels_test_2_5_sec_50ol\
= train_test(2.56, 50, USC_HAD_df, 100)
# 2.5 second window size, 75% overlap
USC_features_train_2_5_sec_75ol, USC_features_test_2_5_sec_75ol, USC_labels_train_2_5_sec_75ol, USC_labels_test_2_5_sec_75ol\
= train_test(2.56, 75, USC_HAD_df, 100)

# UTD-MHAD (50Hz)
# 1 second window size, 25% overlap
UTD_features_train_1_sec_25ol, UTD_features_test_1_sec_25ol, UTD_labels_train_1_sec_25ol, UTD_labels_test_1_sec_25ol =\
train_test(0.98, 25, UTD_MHAD_df, 50)
# 1 second window size, 50% overlap
UTD_features_train_1_sec_50ol, UTD_features_test_1_sec_50ol, UTD_labels_train_1_sec_50ol, UTD_labels_test_1_sec_50ol =\
train_test(0.98, 50, UTD_MHAD_df, 50)
# 1 second window size, 75% overlap
UTD_features_train_1_sec_75ol, UTD_features_test_1_sec_75ol, UTD_labels_train_1_sec_75ol, UTD_labels_test_1_sec_75ol =\
train_test(0.98, 75, UTD_MHAD_df, 50)
# 1.5 second window size, 25% overlap
UTD_features_train_1_5_sec_25ol, UTD_features_test_1_5_sec_25ol, UTD_labels_train_1_5_sec_25ol, UTD_labels_test_1_5_sec_25ol\
= train_test(1.62, 25, UTD_MHAD_df, 50)
# 1.5 second window size, 50% overlap
UTD_features_train_1_5_sec_50ol, UTD_features_test_1_5_sec_50ol, UTD_labels_train_1_5_sec_50ol, UTD_labels_test_1_5_sec_50ol\
= train_test(1.62, 50, UTD_MHAD_df, 50)
# 1.5 second window size, 75% overlap
UTD_features_train_1_5_sec_75ol, UTD_features_test_1_5_sec_75ol, UTD_labels_train_1_5_sec_75ol, UTD_labels_test_1_5_sec_75ol\
= train_test(1.62, 75, UTD_MHAD_df, 50)
# 2 second window size, 25% overlap
UTD_features_train_2_sec_25ol, UTD_features_test_2_sec_25ol, UTD_labels_train_2_sec_25ol, UTD_labels_test_2_sec_25ol =\
train_test(2, 25, UTD_MHAD_df, 50)
# 2 second window size, 50% overlap
UTD_features_train_2_sec_50ol, UTD_features_test_2_sec_50ol, UTD_labels_train_2_sec_50ol, UTD_labels_test_2_sec_50ol =\
train_test(2, 50, UTD_MHAD_df, 50)
# 2 second window size, 75% overlap
UTD_features_train_2_sec_75ol, UTD_features_test_2_sec_75ol, UTD_labels_train_2_sec_75ol, UTD_labels_test_2_sec_75ol =\
train_test(2, 75, UTD_MHAD_df, 50)
# 2.5 second window size, 25% overlap
UTD_features_train_2_5_sec_25ol, UTD_features_test_2_5_sec_25ol, UTD_labels_train_2_5_sec_25ol, UTD_labels_test_2_5_sec_25ol\
= train_test(2.42, 25, UTD_MHAD_df, 50)
# 2.5 second window size, 50% overlap
UTD_features_train_2_5_sec_50ol, UTD_features_test_2_5_sec_50ol, UTD_labels_train_2_5_sec_50ol, UTD_labels_test_2_5_sec_50ol\
= train_test(2.42, 50, UTD_MHAD_df, 50)
# 2.5 second window size, 75% overlap
UTD_features_train_2_5_sec_75ol, UTD_features_test_2_5_sec_75ol, UTD_labels_train_2_5_sec_75ol, UTD_labels_test_2_5_sec_75ol\
= train_test(2.42, 75, UTD_MHAD_df, 50)

# UCI-HAR (50Hz)
# 1 second window size, 25% overlap
UCI_features_train_1_sec_25ol, UCI_features_test_1_sec_25ol, UCI_labels_train_1_sec_25ol, UCI_labels_test_1_sec_25ol =\
train_test(0.98, 25, UCI_HAR_df_proc, 50)
# 1 second window size, 50% overlap
UCI_features_train_1_sec_50ol, UCI_features_test_1_sec_50ol, UCI_labels_train_1_sec_50ol, UCI_labels_test_1_sec_50ol =\
train_test(0.98, 50, UCI_HAR_df_proc, 50)
# 1 second window size, 75% overlap
UCI_features_train_1_sec_75ol, UCI_features_test_1_sec_75ol, UCI_labels_train_1_sec_75ol, UCI_labels_test_1_sec_75ol =\
train_test(0.98, 75, UCI_HAR_df_proc, 50)
# 1.5 second window size, 25% overlap
UCI_features_train_1_5_sec_25ol, UCI_features_test_1_5_sec_25ol, UCI_labels_train_1_5_sec_25ol, UCI_labels_test_1_5_sec_25ol\
= train_test(1.62, 25, UCI_HAR_df_proc, 50)
# 1.5 second window size, 50% overlap
UCI_features_train_1_5_sec_50ol, UCI_features_test_1_5_sec_50ol, UCI_labels_train_1_5_sec_50ol, UCI_labels_test_1_5_sec_50ol\
= train_test(1.62, 50, UCI_HAR_df_proc, 50)
# 1.5 second window size, 75% overlap
UCI_features_train_1_5_sec_75ol, UCI_features_test_1_5_sec_75ol, UCI_labels_train_1_5_sec_75ol, UCI_labels_test_1_5_sec_75ol\
= train_test(1.62, 75, UCI_HAR_df_proc, 50)
# 2 second window size, 25% overlap
UCI_features_train_2_sec_25ol, UCI_features_test_2_sec_25ol, UCI_labels_train_2_sec_25ol, UCI_labels_test_2_sec_25ol =\
train_test(2, 25, UCI_HAR_df_proc, 50)
# 2 second window size, 50% overlap
UCI_features_train_2_sec_50ol, UCI_features_test_2_sec_50ol, UCI_labels_train_2_sec_50ol, UCI_labels_test_2_sec_50ol =\
train_test(2, 50, UCI_HAR_df_proc, 50)
# 2 second window size, 75% overlap
UCI_features_train_2_sec_75ol, UCI_features_test_2_sec_75ol, UCI_labels_train_2_sec_75ol, UCI_labels_test_2_sec_75ol =\
train_test(2, 75, UCI_HAR_df_proc, 50)
# 2.5 second window size, 25% overlap
UCI_features_train_2_5_sec_25ol, UCI_features_test_2_5_sec_25ol, UCI_labels_train_2_5_sec_25ol, UCI_labels_test_2_5_sec_25ol\
= train_test(2.42, 25, UCI_HAR_df_proc, 50)
# 2.5 second window size, 50% overlap
UCI_features_train_2_5_sec_50ol, UCI_features_test_2_5_sec_50ol, UCI_labels_train_2_5_sec_50ol, UCI_labels_test_2_5_sec_50ol\
= train_test(2.42, 50, UCI_HAR_df_proc, 50)
# 2.5 second window size, 75% overlap
UCI_features_train_2_5_sec_75ol, UCI_features_test_2_5_sec_75ol, UCI_labels_train_2_5_sec_75ol, UCI_labels_test_2_5_sec_75ol\
= train_test(2.42, 75, UCI_HAR_df_proc, 50)

In [36]:
# save each dataset
# USC-HAD
np.save(f'Data/Window Data/USC-HAD/USC_features_train_1_sec_25ol.npy', USC_features_train_1_sec_25ol)
np.save(f'Data/Window Data/USC-HAD/USC_features_test_1_sec_25ol.npy', USC_features_test_1_sec_25ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_train_1_sec_25ol.npy', USC_labels_train_1_sec_25ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_test_1_sec_25ol.npy', USC_labels_test_1_sec_25ol)

np.save(f'Data/Window Data/USC-HAD/USC_features_train_1_sec_50ol.npy', USC_features_train_1_sec_50ol)
np.save(f'Data/Window Data/USC-HAD/USC_features_test_1_sec_50ol.npy', USC_features_test_1_sec_50ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_train_1_sec_50ol.npy', USC_labels_train_1_sec_50ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_test_1_sec_50ol.npy', USC_labels_test_1_sec_50ol)

np.save(f'Data/Window Data/USC-HAD/USC_features_train_1_sec_75ol.npy', USC_features_train_1_sec_75ol)
np.save(f'Data/Window Data/USC-HAD/USC_features_test_1_sec_75ol.npy', USC_features_test_1_sec_75ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_train_1_sec_75ol.npy', USC_labels_train_1_sec_75ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_test_1_sec_75ol.npy', USC_labels_test_1_sec_75ol)

np.save(f'Data/Window Data/USC-HAD/USC_features_train_1_5_sec_25ol.npy', USC_features_train_1_5_sec_25ol)
np.save(f'Data/Window Data/USC-HAD/USC_features_test_1_5_sec_25ol.npy', USC_features_test_1_5_sec_25ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_train_1_5_sec_25ol.npy', USC_labels_train_1_5_sec_25ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_test_1_5_sec_25ol.npy', USC_labels_test_1_5_sec_25ol)

np.save(f'Data/Window Data/USC-HAD/USC_features_train_1_5_sec_50ol.npy', USC_features_train_1_5_sec_50ol)
np.save(f'Data/Window Data/USC-HAD/USC_features_test_1_5_sec_50ol.npy', USC_features_test_1_5_sec_50ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_train_1_5_sec_50ol.npy', USC_labels_train_1_5_sec_50ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_test_1_5_sec_50ol.npy', USC_labels_test_1_5_sec_50ol)

np.save(f'Data/Window Data/USC-HAD/USC_features_train_1_5_sec_75ol.npy', USC_features_train_1_5_sec_75ol)
np.save(f'Data/Window Data/USC-HAD/USC_features_test_1_5_sec_75ol.npy', USC_features_test_1_5_sec_75ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_train_1_5_sec_75ol.npy', USC_labels_train_1_5_sec_75ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_test_1_5_sec_75ol.npy', USC_labels_test_1_5_sec_75ol)

np.save(f'Data/Window Data/USC-HAD/USC_features_train_2_sec_25ol.npy', USC_features_train_2_sec_25ol)
np.save(f'Data/Window Data/USC-HAD/USC_features_test_2_sec_25ol.npy', USC_features_test_2_sec_25ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_train_2_sec_25ol.npy', USC_labels_train_2_sec_25ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_test_2_sec_25ol.npy', USC_labels_test_2_sec_25ol)

np.save(f'Data/Window Data/USC-HAD/USC_features_train_2_sec_50ol.npy', USC_features_train_2_sec_50ol)
np.save(f'Data/Window Data/USC-HAD/USC_features_test_2_sec_50ol.npy', USC_features_test_2_sec_50ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_train_2_sec_50ol.npy', USC_labels_train_2_sec_50ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_test_2_sec_50ol.npy', USC_labels_test_2_sec_50ol)

np.save(f'Data/Window Data/USC-HAD/USC_features_train_2_sec_75ol.npy', USC_features_train_2_sec_75ol)
np.save(f'Data/Window Data/USC-HAD/USC_features_test_2_sec_75ol.npy', USC_features_test_2_sec_75ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_train_2_sec_75ol.npy', USC_labels_train_2_sec_75ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_test_2_sec_75ol.npy', USC_labels_test_2_sec_75ol)

np.save(f'Data/Window Data/USC-HAD/USC_features_train_2_5_sec_25ol.npy', USC_features_train_2_5_sec_25ol)
np.save(f'Data/Window Data/USC-HAD/USC_features_test_2_5_sec_25ol.npy', USC_features_test_2_5_sec_25ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_train_2_5_sec_25ol.npy', USC_labels_train_2_5_sec_25ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_test_2_5_sec_25ol.npy', USC_labels_test_2_5_sec_25ol)

np.save(f'Data/Window Data/USC-HAD/USC_features_train_2_5_sec_50ol.npy', USC_features_train_2_5_sec_50ol)
np.save(f'Data/Window Data/USC-HAD/USC_features_test_2_5_sec_50ol.npy', USC_features_test_2_5_sec_50ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_train_2_5_sec_50ol.npy', USC_labels_train_2_5_sec_50ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_test_2_5_sec_50ol.npy', USC_labels_test_2_5_sec_50ol)

np.save(f'Data/Window Data/USC-HAD/USC_features_train_2_5_sec_75ol.npy', USC_features_train_2_5_sec_75ol)
np.save(f'Data/Window Data/USC-HAD/USC_features_test_2_5_sec_75ol.npy', USC_features_test_2_5_sec_75ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_train_2_5_sec_75ol.npy', USC_labels_train_2_5_sec_75ol)
np.save(f'Data/Window Data/USC-HAD/USC_labels_test_2_5_sec_75ol.npy', USC_labels_test_2_5_sec_75ol)

# UTD-MHAD
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_train_1_sec_25ol.npy', UTD_features_train_1_sec_25ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_test_1_sec_25ol.npy', UTD_features_test_1_sec_25ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_train_1_sec_25ol.npy', UTD_labels_train_1_sec_25ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_test_1_sec_25ol.npy', UTD_labels_test_1_sec_25ol)

np.save(f'Data/Window Data/UTD-MHAD/UTD_features_train_1_sec_50ol.npy', UTD_features_train_1_sec_50ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_test_1_sec_50ol.npy', UTD_features_test_1_sec_50ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_train_1_sec_50ol.npy', UTD_labels_train_1_sec_50ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_test_1_sec_50ol.npy', UTD_labels_test_1_sec_50ol)

np.save(f'Data/Window Data/UTD-MHAD/UTD_features_train_1_sec_75ol.npy', UTD_features_train_1_sec_75ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_test_1_sec_75ol.npy', UTD_features_test_1_sec_75ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_train_1_sec_75ol.npy', UTD_labels_train_1_sec_75ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_test_1_sec_75ol.npy', UTD_labels_test_1_sec_75ol)

np.save(f'Data/Window Data/UTD-MHAD/UTD_features_train_1_5_sec_25ol.npy', UTD_features_train_1_5_sec_25ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_test_1_5_sec_25ol.npy', UTD_features_test_1_5_sec_25ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_train_1_5_sec_25ol.npy', UTD_labels_train_1_5_sec_25ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_test_1_5_sec_25ol.npy', UTD_labels_test_1_5_sec_25ol)

np.save(f'Data/Window Data/UTD-MHAD/UTD_features_train_1_5_sec_50ol.npy', UTD_features_train_1_5_sec_50ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_test_1_5_sec_50ol.npy', UTD_features_test_1_5_sec_50ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_train_1_5_sec_50ol.npy', UTD_labels_train_1_5_sec_50ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_test_1_5_sec_50ol.npy', UTD_labels_test_1_5_sec_50ol)

np.save(f'Data/Window Data/UTD-MHAD/UTD_features_train_1_5_sec_75ol.npy', UTD_features_train_1_5_sec_75ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_test_1_5_sec_75ol.npy', UTD_features_test_1_5_sec_75ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_train_1_5_sec_75ol.npy', UTD_labels_train_1_5_sec_75ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_test_1_5_sec_75ol.npy', UTD_labels_test_1_5_sec_75ol)

np.save(f'Data/Window Data/UTD-MHAD/UTD_features_train_2_sec_25ol.npy', UTD_features_train_2_sec_25ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_test_2_sec_25ol.npy', UTD_features_test_2_sec_25ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_train_2_sec_25ol.npy', UTD_labels_train_2_sec_25ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_test_2_sec_25ol.npy', UTD_labels_test_2_sec_25ol)

np.save(f'Data/Window Data/UTD-MHAD/UTD_features_train_2_sec_50ol.npy', UTD_features_train_2_sec_50ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_test_2_sec_50ol.npy', UTD_features_test_2_sec_50ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_train_2_sec_50ol.npy', UTD_labels_train_2_sec_50ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_test_2_sec_50ol.npy', UTD_labels_test_2_sec_50ol)

np.save(f'Data/Window Data/UTD-MHAD/UTD_features_train_2_sec_75ol.npy', UTD_features_train_2_sec_75ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_test_2_sec_75ol.npy', UTD_features_test_2_sec_75ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_train_2_sec_75ol.npy', UTD_labels_train_2_sec_75ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_test_2_sec_75ol.npy', UTD_labels_test_2_sec_75ol)

np.save(f'Data/Window Data/UTD-MHAD/UTD_features_train_2_5_sec_25ol.npy', UTD_features_train_2_5_sec_25ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_test_2_5_sec_25ol.npy', UTD_features_test_2_5_sec_25ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_train_2_5_sec_25ol.npy', UTD_labels_train_2_5_sec_25ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_test_2_5_sec_25ol.npy', UTD_labels_test_2_5_sec_25ol)

np.save(f'Data/Window Data/UTD-MHAD/UTD_features_train_2_5_sec_50ol.npy', UTD_features_train_2_5_sec_50ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_test_2_5_sec_50ol.npy', UTD_features_test_2_5_sec_50ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_train_2_5_sec_50ol.npy', UTD_labels_train_2_5_sec_50ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_test_2_5_sec_50ol.npy', UTD_labels_test_2_5_sec_50ol)

np.save(f'Data/Window Data/UTD-MHAD/UTD_features_train_2_5_sec_75ol.npy', UTD_features_train_2_5_sec_75ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_features_test_2_5_sec_75ol.npy', UTD_features_test_2_5_sec_75ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_train_2_5_sec_75ol.npy', UTD_labels_train_2_5_sec_75ol)
np.save(f'Data/Window Data/UTD-MHAD/UTD_labels_test_2_5_sec_75ol.npy', UTD_labels_test_2_5_sec_75ol)

# UCI-HAR
np.save(f'Data/Window Data/UCI-HAR/UCI_features_train_1_sec_25ol.npy', UCI_features_train_1_sec_25ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_features_test_1_sec_25ol.npy', UCI_features_test_1_sec_25ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_train_1_sec_25ol.npy', UCI_labels_train_1_sec_25ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_test_1_sec_25ol.npy', UCI_labels_test_1_sec_25ol)

np.save(f'Data/Window Data/UCI-HAR/UCI_features_train_1_sec_50ol.npy', UCI_features_train_1_sec_50ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_features_test_1_sec_50ol.npy', UCI_features_test_1_sec_50ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_train_1_sec_50ol.npy', UCI_labels_train_1_sec_50ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_test_1_sec_50ol.npy', UCI_labels_test_1_sec_50ol)

np.save(f'Data/Window Data/UCI-HAR/UCI_features_train_1_sec_75ol.npy', UCI_features_train_1_sec_75ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_features_test_1_sec_75ol.npy', UCI_features_test_1_sec_75ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_train_1_sec_75ol.npy', UCI_labels_train_1_sec_75ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_test_1_sec_75ol.npy', UCI_labels_test_1_sec_75ol)

np.save(f'Data/Window Data/UCI-HAR/UCI_features_train_1_5_sec_25ol.npy', UCI_features_train_1_5_sec_25ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_features_test_1_5_sec_25ol.npy', UCI_features_test_1_5_sec_25ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_train_1_5_sec_25ol.npy', UCI_labels_train_1_5_sec_25ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_test_1_5_sec_25ol.npy', UCI_labels_test_1_5_sec_25ol)

np.save(f'Data/Window Data/UCI-HAR/UCI_features_train_1_5_sec_50ol.npy', UCI_features_train_1_5_sec_50ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_features_test_1_5_sec_50ol.npy', UCI_features_test_1_5_sec_50ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_train_1_5_sec_50ol.npy', UCI_labels_train_1_5_sec_50ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_test_1_5_sec_50ol.npy', UCI_labels_test_1_5_sec_50ol)

np.save(f'Data/Window Data/UCI-HAR/UCI_features_train_1_5_sec_75ol.npy', UCI_features_train_1_5_sec_75ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_features_test_1_5_sec_75ol.npy', UCI_features_test_1_5_sec_75ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_train_1_5_sec_75ol.npy', UCI_labels_train_1_5_sec_75ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_test_1_5_sec_75ol.npy', UCI_labels_test_1_5_sec_75ol)

np.save(f'Data/Window Data/UCI-HAR/UCI_features_train_2_sec_25ol.npy', UCI_features_train_2_sec_25ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_features_test_2_sec_25ol.npy', UCI_features_test_2_sec_25ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_train_2_sec_25ol.npy', UCI_labels_train_2_sec_25ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_test_2_sec_25ol.npy', UCI_labels_test_2_sec_25ol)

np.save(f'Data/Window Data/UCI-HAR/UCI_features_train_2_sec_50ol.npy', UCI_features_train_2_sec_50ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_features_test_2_sec_50ol.npy', UCI_features_test_2_sec_50ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_train_2_sec_50ol.npy', UCI_labels_train_2_sec_50ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_test_2_sec_50ol.npy', UCI_labels_test_2_sec_50ol)

np.save(f'Data/Window Data/UCI-HAR/UCI_features_train_2_sec_75ol.npy', UCI_features_train_2_sec_75ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_features_test_2_sec_75ol.npy', UCI_features_test_2_sec_75ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_train_2_sec_75ol.npy', UCI_labels_train_2_sec_75ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_test_2_sec_75ol.npy', UCI_labels_test_2_sec_75ol)

np.save(f'Data/Window Data/UCI-HAR/UCI_features_train_2_5_sec_25ol.npy', UCI_features_train_2_5_sec_25ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_features_test_2_5_sec_25ol.npy', UCI_features_test_2_5_sec_25ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_train_2_5_sec_25ol.npy', UCI_labels_train_2_5_sec_25ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_test_2_5_sec_25ol.npy', UCI_labels_test_2_5_sec_25ol)

np.save(f'Data/Window Data/UCI-HAR/UCI_features_train_2_5_sec_50ol.npy', UCI_features_train_2_5_sec_50ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_features_test_2_5_sec_50ol.npy', UCI_features_test_2_5_sec_50ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_train_2_5_sec_50ol.npy', UCI_labels_train_2_5_sec_50ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_test_2_5_sec_50ol.npy', UCI_labels_test_2_5_sec_50ol)

np.save(f'Data/Window Data/UCI-HAR/UCI_features_train_2_5_sec_75ol.npy', UCI_features_train_2_5_sec_75ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_features_test_2_5_sec_75ol.npy', UCI_features_test_2_5_sec_75ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_train_2_5_sec_75ol.npy', UCI_labels_train_2_5_sec_75ol)
np.save(f'Data/Window Data/UCI-HAR/UCI_labels_test_2_5_sec_75ol.npy', UCI_labels_test_2_5_sec_75ol)